# 📓 06_transform_playlists_static.ipynb
Transformar playlists.json en una tabla limpia de playlists manuales, excluyendo uploads, lista para análisis y joins.

In [ ]:
from pathlib import Path
import json
import pandas as pd
from datetime import datetime

In [ ]:
PROJECT_ROOT = Path.cwd().parents[0]

RAW_PATH = PROJECT_ROOT / "data" / "raw" / "youtube"
PROCESSED_PATH = PROJECT_ROOT / "data" / "processed" / "youtube"

PROCESSED_PATH.mkdir(parents=True, exist_ok=True)


In [ ]:
with open(RAW_PATH / "playlists.json", "r", encoding="utf-8") as f:
    playlists_raw = json.load(f)

len(playlists_raw["items"])


In [ ]:
UPLOADS_PLAYLIST_ID = "UUUEOHBht8pnQhQvCfIcl-gg"

In [ ]:
from datetime import datetime, timezone

# Timestamp explícitamente en UTC
EXTRACTED_AT = datetime.now(timezone.utc)

EXTRACTED_AT

In [ ]:
rows = []

for item in playlists_raw["items"]:
    playlist_id = item["id"]

    # Excluir uploads
    if playlist_id == UPLOADS_PLAYLIST_ID:
        continue

    snippet = item["snippet"]
    content = item.get("contentDetails", {})
    status = item.get("status", {})

    rows.append({
        "playlist_id": playlist_id,
        "channel_id": snippet.get("channelId"),
        "title": snippet.get("title"),
        "description": snippet.get("description", ""),
        "item_count": content.get("itemCount", 0),
        "privacy_status": status.get("privacyStatus"),
        "published_at": snippet.get("publishedAt"),
        "thumbnail_url": (
            snippet.get("thumbnails", {})
                   .get("high", {})
                   .get("url")
        ),
        "playlist_url": f"https://www.youtube.com/playlist?list={playlist_id}",
        "extracted_at": EXTRACTED_AT
    })



In [ ]:
df_playlists_manual_static = pd.DataFrame(rows)

df_playlists_manual_static.shape


In [ ]:
df_playlists_manual_static.dtypes

In [ ]:
# Convertir el published_at a fecha
df_playlists_manual_static["published_at"] = (
    pd.to_datetime(df_playlists_manual_static["published_at"], utc=True)
)

In [ ]:
df_playlists_manual_static.dtypes

In [ ]:
df_playlists_manual_static = df_playlists_manual_static[
    [
        "playlist_id",
        "channel_id",
        "title",
        "description",
        "item_count",
        "privacy_status",
        "published_at",
        "thumbnail_url",
        "playlist_url",
        "extracted_at",
    ]
]



> ⚠️ Nota (fase de desarrollo)
>
> El guardado en formato Parquet se utiliza temporalmente para pruebas y separación entre notebooks.
> En la versión final del pipeline (.py), este paso será omitido y el DataFrame se enviará directamente a BigQuery, donde residirá el histórico definitivo.

In [ ]:
output_file = PROCESSED_PATH / "playlists_manual_static.parquet"

df_playlists_manual_static.to_parquet(output_file, index=False)

output_file

In [ ]:
from pathlib import Path
import pandas as pd

PROJECT_ROOT = Path.cwd().parents[0]
PROCESSED_PATH = PROJECT_ROOT / "data" / "processed" / "youtube"

df_playlists_manual_static = pd.read_parquet(
    PROCESSED_PATH / "playlists_manual_static.parquet"
)

df_playlists_manual_static.head()


In [ ]:
df_playlists_manual_static.shape